# Neural Network (NN) 
### This notebook uses a Neural Network algorithims to find the best predition model for this data set. 

<br/>Data Source: 
https://archive.ics.uci.edu/ml/datasets/SGEMM+GPU+kernel+performance

In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import learning_curve
import datetime


seed = 20200309

file1 = r'sgemm_product.csv'

## Read Data in and Prep it

In [2]:
# Read and prepare sgemm_product data 
df_one_01 = pd.read_csv(file1)
print(df_one_01.shape)

#Add an average of the Runs
df_one_01['RunAvg'] = (df_one_01['Run1 (ms)']+df_one_01['Run2 (ms)']+df_one_01['Run3 (ms)']+df_one_01['Run4 (ms)'])/4
print(df_one_01.shape,'Added Average Column')

df_one_01['B_RunAvg'] = np.where(df_one_01['RunAvg'] < df_one_01['RunAvg'].mean(), 0, 1)
print(df_one_01.shape,'Turned RunAvg into a Binary Class')

#Drop columns not needed
df_one_01.drop(labels=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)','RunAvg']
            , axis=1
            , inplace=True)
print(df_one_01.shape,'Data has been read in.')
print(df_one_01.head())

(241600, 18)


(241600, 19) Added Average Column
(241600, 20) Turned RunAvg into a Binary Class
(241600, 15) Data has been read in.
   MWG  NWG  KWG  MDIMC  NDIMC  MDIMA  NDIMB  KWI  VWM  VWN  STRM  STRN  SA  \
0   16   16   16      8      8      8      8    2    1    1     0     0   0   
1   16   16   16      8      8      8      8    2    1    1     0     0   0   
2   16   16   16      8      8      8      8    2    1    1     0     0   1   
3   16   16   16      8      8      8      8    2    1    1     0     0   1   
4   16   16   16      8      8      8      8    2    1    1     0     1   0   

   SB  B_RunAvg  
0   0         0  
1   1         0  
2   0         0  
3   1         0  
4   0         0  


## Split Train and Test Data sets

In [3]:
# ==============================================================================
# Create a Training and Testing data set
# ==============================================================================
X = df_one_01.drop(['B_RunAvg'], axis=1)
y = df_one_01['B_RunAvg']

d1_X_train, d1_X_test, d1_y_train, d1_y_test = train_test_split(X, y, test_size=0.30, random_state=seed)

print('d1_X_train\t',d1_X_train.shape,'\t', len(d1_X_train)/len(df_one_01))
print('d1_y_train\t',d1_y_train.shape,'\t', len(d1_y_train)/len(df_one_01))
print('d1_X_test\t',d1_X_test.shape,'\t', len(d1_X_test)/len(df_one_01))
print('d1_y_test\t',d1_y_test.shape,'\t', len(d1_y_test)/len(df_one_01),'\n')

d1_X_train	 (169120, 14) 	 0.7
d1_y_train	 (169120,) 	 0.7
d1_X_test	 (72480, 14) 	 0.3
d1_y_test	 (72480,) 	 0.3 



## Train Neural Network and test the model

In [6]:
%%time

NN_1 = MLPClassifier(hidden_layer_sizes=(14,14),activation='tanh') # Options: 'identity', 'logistic', 'tanh', 'relu'
NN_1.fit(d1_X_train,d1_y_train)
NN_Pred_y = mlp_1.predict(d1_X_test)

NN_Confusion_Matrix = pd.DataFrame(data=confusion_matrix(d1_y_test,NN_Pred_y)
                                    , index =['Actual No', 'Actual Yes']
                                    , columns =['Prediced No','Predicted Yes'])
NN_CR=classification_report(d1_y_test,NN_Pred_y)
MSE_NN = (np.square(d1_y_test - NN_Pred_y)).mean()
SSE_NN = (np.square(d1_y_test - NN_Pred_y)).sum()

print('NN MSE',MSE_NN)
print('NN SSE',SSE_NN)
print(NN_Confusion_Matrix,'\n')
print(NN_CR,'\n\n')

NN MSE 0.029373620309050774
NN SSE 2129
            Prediced No  Predicted Yes
Actual No         52343           1081
Actual Yes         1048          18008 

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     53424
           1       0.94      0.95      0.94     19056

    accuracy                           0.97     72480
   macro avg       0.96      0.96      0.96     72480
weighted avg       0.97      0.97      0.97     72480
 


Wall time: 1min 58s


### The Neural Network model had an accuracy of .97